# Question 1a:

### Write a function to calculate the change in computation time and prediction accuracy per principal component added to the feature set for kNN. Have your function call be pcaCompare(X,Yrows,k,pcs), where X, Y, and k are the usual inputs to kNN, rows is the rows of the data we want to test, and pcs is the maximum number of principal components tested. Have it return a 3 column matrix representing the change in time and accuracy (not as a percentage) that occurs by adding this PC as a feature (eg. negative indicates the value went down when it was added) in the first and second columns, respectively, and the variance of the PC in the third, where the PC number is the row number.


In [ ]:
# import drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def explained_variance(pca):
  pca_ratio = (pca.explained_variance_ratio_).tolist()
  total_variance = 0
  for number in pca_ratio:
    total_variance += number
  return(total_variance)

def random_samples_generator(X, Y, rows):
  import random
  random_indexes = random.sample(range(0, len(Y)), rows)   # randomly select some indexes (sample b/c no repeats)
  newX = []
  newY = []
  for index in random_indexes:
    newX.append(X.iloc[index])    # return a matrix of the Xs at the random indexes
    newY.append(Y.iloc[index])    # return a list of the Ys at the random indexes 
  return pd.DataFrame(newX), (newY)

def pcsCompare(X,Y,rows,k,pcs):
  # X = train_set_X, Yrows = train_set_Y, K = maxK to test, rows = test set, psc = maximum principle components tested 
  from sklearn.neighbors import KNeighborsRegressor
  from sklearn.decomposition import PCA
  import statistics 
  import time
  import random

  newx, newy = random_samples_generator(X, Y, rows)  # randomly choose some 'rows' rows because the data is ordered 

  PC_dict = {}
  for i in range(pcs):   # for each number of principle components
    pca = PCA(n_components=(i+1))  # generate a PCA with that many components (remember 0 indexing)
    pca.fit(X)
    total_variance = explained_variance(pca)   # get the amount of variance that is explained total with this many components
    X_pca = pca.transform(X)
    newx_pca = pca.transform(newx)    
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_pca, Y)    # fit the knn to a regression equation with that many components
    start_time = time.time()
    preds = (knn.predict(newx_pca).round())    # generate a prediction and see how long this takes 
    total_time = time.time() - start_time
    differences = []
    correct_instances = 0
    for j in range(rows):   # compare each test row with its correct Y value 
      real_Y = newy[j]
      Y_pred = preds[j]
      if real_Y == Y_pred:
        correct_instances += 1
    accuracy = round(correct_instances/rows, 4)
    PC_dict[i+1] = [accuracy, total_time, total_variance]
  PC_change_matrix = pd.DataFrame.from_dict(PC_dict, orient='index', columns=['Accuracy', 'Time', 'Variance'])
  return(PC_change_matrix)


## Functions used to test the pcsCompre function:

In [ ]:
import pandas as pd
import numpy as np
my_path = '/content/drive/My Drive/YearPredictionMSD.txt'
ms = pd.read_csv(my_path, header=None)

In [ ]:
X = ms.iloc[:,1:]
Y = ms.iloc[:,0]
rows = 10
k = 20 
pcs = 10

pcs = pcsCompare(X, Y, rows, k, pcs)
pcs


,Accuracy,Time,Variance
1,0.0,0.001385,0.469129
2,0.0,0.001148,0.613581
3,0.0,0.001341,0.705189
4,0.1,0.001458,0.754606
5,0.0,0.001838,0.785573
6,0.1,0.002609,0.808269
7,0.1,0.003489,0.826119
8,0.1,0.005265,0.842415
9,0.1,0.006860,0.857915
10,0.0,0.008617,0.870571
